<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/URL_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install unidecode
!pip install word2number
!pip install contractions

     |████████████████████████████████| 235 kB 3.3 MB/s 
  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5582 sha256=2d0fd7c8199e211b0363d118d854e410e428d9dace2a975469046c40515b677a
  Stored in directory: /root/.cache/pip/wheels/4b/c3/77/a5f48aeb0d3efb7cd5ad61cbd3da30bbf9ffc9662b07c9f879
Successfully built word2number
     |████████████████████████████████| 284 kB 3.8 MB/s 
     |████████████████████████████████| 321 kB 53.2 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85449 sha256=49602888396632a3f56dbb1b395393bc04579373bf95a5f0d9bbea12936a5ac7
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [5]:
import pandas as pd
import numpy as np
import nltk
import gensim
from gensim.models import Word2Vec
import string
string.punctuation
nltk.download('stopwords')
import re
from bs4 import BeautifulSoup
import spacy
import unidecode
from word2number import w2n
import contractions
from nltk.tokenize.treebank import TreebankWordDetokenizer
# load spacy model, can be "en_core_web_sm" as well
from sklearn.model_selection import train_test_split
import xgboost as xgb



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
from google.colab import drive
drive._mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
df_fake = pd.read_csv('/content/drive/MyDrive/Thesis/gossipcop_fake.csv')
df_real = pd.read_csv('/content/drive/MyDrive/Thesis/gossipcop_real.csv')
df_fake['labelML']=0
df_real['labelML']=1

df_fake.count()


id           5323
news_url     5067
title        5323
tweet_ids    5135
labelML      5323
dtype: int64

In [ ]:
#dforiginal = df_fake.head(5).append(df_real.head(5))

In [8]:
df= df_fake.head(10).append(df_real.head(10))
#df= df_fake.append(df_real)
#df.reset_index(drop=True, inplace=True)

In [9]:
#print(df.columns)
##df.count()
df

,id,news_url,title,tweet_ids,labelML
0,gossipcop-2493749932,www.dailymail.co.uk/tvshowbiz/article-5874213/...,Did Miley Cyrus and Liam Hemsworth secretly ge...,284329075902926848\t284332744559968256\t284335...,0
1,gossipcop-4580247171,hollywoodlife.com/2018/05/05/paris-jackson-car...,Paris Jackson & Cara Delevingne Enjoy Night Ou...,992895508267130880\t992897935418503169\t992899...,0
2,gossipcop-941805037,variety.com/2017/biz/news/tax-march-donald-tru...,Celebrities Join Tax March in Protest of Donal...,853359353532829696\t853359576543920128\t853359...,0
3,gossipcop-2547891536,www.dailymail.co.uk/femail/article-3499192/Do-...,Cindy Crawford's daughter Kaia Gerber wears a ...,988821905196158981\t988824206556172288\t988825...,0
4,gossipcop-5476631226,variety.com/2018/film/news/list-2018-oscar-nom...,Full List of 2018 Oscar Nominations – Variety,955792793632432131\t955795063925301249\t955798...,0
5,gossipcop-5189580095,www.townandcountrymag.com/society/tradition/a1...,Here's What Really Happened When JFK Jr. Met P...,890253005299351552\t890401381814870016\t890491...,0
6,gossipcop-9588339534,www.foxnews.com/entertainment/2016/12/16/bigge...,Biggest celebrity scandals of 2016,683226380742557696\t748604615503929345\t748604...,0
7,gossipcop-8753274298,www.eonline.com/news/958257/caitlyn-jenner-add...,Caitlyn Jenner Addresses Rumored Romance With ...,1026891446081728512\t1026891745219543043\t1026...,0
8,gossipcop-8105333868,www.inquisitr.com/3871816/taylor-swift-reporte...,Taylor Swift Reportedly Reacts To Tom Hiddlest...,818928533569437697\t819100640878202880\t819174...,0
9,gossipcop-2803748870,www.huffingtonpost.com/entry/kate-mckinnon-the...,"For The Love Of God, Why Can't Anyone Write Ka...",816030248190046212\t816030859484626947\t816049...,0


In [10]:
print(df)

                     id  ... labelML
0  gossipcop-2493749932  ...       0
1  gossipcop-4580247171  ...       0
2   gossipcop-941805037  ...       0
3  gossipcop-2547891536  ...       0
4  gossipcop-5476631226  ...       0
5  gossipcop-5189580095  ...       0
6  gossipcop-9588339534  ...       0
7  gossipcop-8753274298  ...       0
8  gossipcop-8105333868  ...       0
9  gossipcop-2803748870  ...       0
0      gossipcop-882573  ...       1
1      gossipcop-875924  ...       1
2      gossipcop-894416  ...       1
3      gossipcop-857248  ...       1
4      gossipcop-884684  ...       1
5      gossipcop-899309  ...       1
6      gossipcop-848092  ...       1
7      gossipcop-897293  ...       1
8      gossipcop-926957  ...       1
9      gossipcop-930832  ...       1

[20 rows x 5 columns]


In [11]:
df1= df.drop(['tweet_ids', 'title','id'], axis=1)
df1.count()

news_url    20
labelML     20
dtype: int64

In [12]:
# FOR URL Preprocessing now. 
dfurl = df.drop(['title', 'tweet_ids','id'], axis=1)
dfurl.head()

,news_url,labelML
0,www.dailymail.co.uk/tvshowbiz/article-5874213/...,0
1,hollywoodlife.com/2018/05/05/paris-jackson-car...,0
2,variety.com/2017/biz/news/tax-march-donald-tru...,0
3,www.dailymail.co.uk/femail/article-3499192/Do-...,0
4,variety.com/2018/film/news/list-2018-oscar-nom...,0


In [13]:
!pip install tldextract
import tldextract

def extractDomain(url):
    if "http" in str(url) or "www" in str(url):
        parsed = tldextract.extract(url)
        parsed = ".".join([i for i in parsed if i])
        return parsed
    else: return str(url)

op = open("out.txt",'w')
# with open("test.txt") as ptr:
#   for lines in ptr.read().split("\n"):
#       op.write(str(extractDomain(lines)) + "\n")

print(extractDomain("https://test.pythonhosted.org/Flask-Mail/"))

test.pythonhosted.org


In [14]:
dfurl

,news_url,labelML
0,www.dailymail.co.uk/tvshowbiz/article-5874213/...,0
1,hollywoodlife.com/2018/05/05/paris-jackson-car...,0
2,variety.com/2017/biz/news/tax-march-donald-tru...,0
3,www.dailymail.co.uk/femail/article-3499192/Do-...,0
4,variety.com/2018/film/news/list-2018-oscar-nom...,0
5,www.townandcountrymag.com/society/tradition/a1...,0
6,www.foxnews.com/entertainment/2016/12/16/bigge...,0
7,www.eonline.com/news/958257/caitlyn-jenner-add...,0
8,www.inquisitr.com/3871816/taylor-swift-reporte...,0
9,www.huffingtonpost.com/entry/kate-mckinnon-the...,0


In [15]:
dfurl['news_url'] = dfurl['news_url'].apply(lambda x: ' '.join(extractDomain(x)))
dfurl

,news_url,labelML
0,w w w . d a i l y m a i l . c o . u k,0
1,h o l l y w o o d l i f e . c o m / 2 0 1 8 / ...,0
2,v a r i e t y . c o m / 2 0 1 7 / b i z / n e ...,0
3,w w w . d a i l y m a i l . c o . u k,0
4,v a r i e t y . c o m / 2 0 1 8 / f i l m / n ...,0
5,w w w . t o w n a n d c o u n t r y m a g . c o m,0
6,w w w . f o x n e w s . c o m,0
7,w w w . e o n l i n e . c o m,0
8,w w w . i n q u i s i t r . c o m,0
9,w w w . h u f f i n g t o n p o s t . c o m,0


In [ ]:
# NLP Preprocessing
from gensim.utils import simple_preprocess

# NLP Preprocess - gensim.utils.simple_preprocess(doc, deacc=False, min_len=2, max_len=15)[source]
# Convert a document into a list of tokens.
# This lowercases, tokenizes, de-accents (optional). – the output are final tokens = unicode strings, that won’t be processed any further.

df1.iloc[:, 0] = df1.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))

In [ ]:
print(df1)
#df1.count()

                                                   title  labelML
0      did miley cyrus and liam hemsworth secretly ge...        0
1      paris jackson cara delevingne enjoy night out ...        0
2      celebrities join tax march in protest of donal...        0
3      cindy crawford daughter kaia gerber wears wig ...        0
4                 full list of oscar nominations variety        0
...                                                  ...      ...
22135  hollywood film awards the complete list of win...        1
22136  jada pinkett smith explains why son jaden move...        1
22137  tinsley mortimer reacts to luann de lesseps re...        1
22138  prince harry carries on princess diana legacy ...        1
22139  kylie jenner is actually terrified of butterflies        1

[22140 rows x 2 columns]


In [ ]:
print(df1)

                                                   title  labelML
0      did miley cyrus and liam hemsworth secretly ge...        0
1      paris jackson cara delevingne enjoy night out ...        0
2      celebrities join tax march in protest of donal...        0
3      cindy crawford daughter kaia gerber wears wig ...        0
4                 full list of oscar nominations variety        0
...                                                  ...      ...
22135  hollywood film awards the complete list of win...        1
22136  jada pinkett smith explains why son jaden move...        1
22137  tinsley mortimer reacts to luann de lesseps re...        1
22138  prince harry carries on princess diana legacy ...        1
22139  kylie jenner is actually terrified of butterflies        1

[22140 rows x 2 columns]


In [ ]:
df1.count()

title      22140
labelML    22140
dtype: int64

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
!gunzip cc.en.300.bin.gz

--2022-02-02 07:41:24--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘cc.en.300.bin.gz’

cc.en.300.bin.gz    100%[===================>]   4.19G  25.4MB/s    in 2m 51s  

2022-02-02 07:44:15 (25.2 MB/s) - ‘cc.en.300.bin.gz’ saved [4503593528/4503593528]

gzip: cc.en.300.bin already exists; do you wish to overwrite (y or n)? n
	not overwritten


trying to see similarity of vectors created.

In [ ]:
!pip install fasttext
!pip install pyfasttext 

In [ ]:
!pip install fasttext
import fasttext as ft

!pip3 install pyfasttext
from pyfasttext import FastText

# Loading model for English
modelog = ft.load_model('cc.en.300.bin') 

In [ ]:
c = modelog.get_sentence_vector('paris hiltion liam hemswoth go to hospital')
d = modelog.get_sentence_vector('miley cyrus liam hemswoth go to hospital')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
cosine_similarity = 1 - spatial.distance.cosine(c, d)
cosine_similarity

0.8526310920715332

In [ ]:
modelog.get_nearest_neighbors('hello')

[(0.7143728733062744, 'hellow'),
 (0.7095366716384888, 'hello.'),
 (0.703833818435669, 'hi'),
 (0.6944323182106018, 'hullo'),
 (0.6912142038345337, 'hello-'),
 (0.663975715637207, 'Hello'),
 (0.6563249230384827, 'hello.I'),
 (0.6529381275177002, 'howdy'),
 (0.6283847689628601, 'hellooo'),
 (0.6235803365707397, 'hellooooo')]

In [ ]:
embedding=[]
for line in df1['title']:
    pred_label=modelog.get_sentence_vector(line)
    embedding.append(pred_label)

# you add the list to the dataframe, then save the datframe to new csv
df1['prediction']=embedding
df1.to_csv('csv_word_embed_gossipcop.csv',index=False)

In [ ]:
df1

,title,labelML,prediction
0,did miley cyrus and liam hemsworth secretly ge...,0,"[0.00040790025, -0.0069974544, 0.043367695, 0...."
1,paris jackson cara delevingne enjoy night out ...,0,"[0.0054201577, -0.003211912, 0.01313437, 0.060..."
2,celebrities join tax march in protest of donal...,0,"[-0.021778336, -0.007887523, -0.012167458, 0.0..."
3,cindy crawford daughter kaia gerber wears wig ...,0,"[0.021224087, 0.003118416, -0.035745192, 0.044..."
4,full list of oscar nominations variety,0,"[-0.04263019, 0.028402044, -0.02406443, 0.0112..."
...,...,...,...
22135,hollywood film awards the complete list of win...,1,"[-0.042594157, 0.03911372, -0.010765053, 0.010..."
22136,jada pinkett smith explains why son jaden move...,1,"[0.0073598004, 0.0268158, 0.027973248, 0.02941..."
22137,tinsley mortimer reacts to luann de lesseps re...,1,"[0.0011494784, 0.00631718, -0.047285415, 0.044..."
22138,prince harry carries on princess diana legacy ...,1,"[-0.020925144, 0.0004961334, 0.012408494, 0.03..."


In [ ]:
type(df1)

pandas.core.frame.DataFrame

In [ ]:
df3 = df1.drop(['title'], axis=1)
df3

,labelML,prediction
0,0,"[0.00040790025, -0.0069974544, 0.043367695, 0...."
1,0,"[0.0054201577, -0.003211912, 0.01313437, 0.060..."
2,0,"[-0.021778336, -0.007887523, -0.012167458, 0.0..."
3,0,"[0.021224087, 0.003118416, -0.035745192, 0.044..."
4,0,"[-0.04263019, 0.028402044, -0.02406443, 0.0112..."
...,...,...
22135,1,"[-0.042594157, 0.03911372, -0.010765053, 0.010..."
22136,1,"[0.0073598004, 0.0268158, 0.027973248, 0.02941..."
22137,1,"[0.0011494784, 0.00631718, -0.047285415, 0.044..."
22138,1,"[-0.020925144, 0.0004961334, 0.012408494, 0.03..."


In [ ]:
#to split the embedding 
for index, arr in df3['prediction'].iteritems():
  for i in range(len(arr)):
    df3.loc[index, str(i)] = arr[i]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


In [ ]:
df3.to_pickle('ourpickle.pkl', compression=None)
df3.to_csv('title_embed_gossipcop.csv', index=False)
#to keep the files for reuse later. 

In [ ]:
df3

,labelML,prediction,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,0,"[0.00040790025, -0.0069974544, 0.043367695, 0....",0.000408,-0.006997,0.043368,0.059105,0.005750,-0.048923,-0.023493,-0.012002,0.008741,0.037747,-0.016657,0.021009,-0.000813,-0.005880,0.021754,0.012577,-0.026488,0.006688,-0.065610,0.031214,0.000651,0.027043,0.026573,-0.008683,0.011755,-0.033109,-0.012075,0.001186,-0.013979,0.112681,0.002063,-0.028648,-0.023800,0.026196,-0.012809,0.025506,0.010329,0.047436,...,-0.010071,0.012518,0.041530,0.023292,-0.059176,0.013486,0.009417,0.022773,-0.007663,-0.022772,0.013969,0.001931,-0.034058,0.000787,0.023271,0.028486,0.013586,0.034805,-0.002466,0.012882,-0.038919,-0.005540,-0.085419,-0.050894,0.005842,0.044290,0.008440,-0.039907,-0.004749,0.006133,0.010734,0.009146,-0.105409,0.021976,-0.012046,0.027996,0.003418,0.054863,-0.031841,-0.042299
1,0,"[0.0054201577, -0.003211912, 0.01313437, 0.060...",0.005420,-0.003212,0.013134,0.060644,-0.007347,-0.006692,0.008933,-0.000213,0.008129,0.027304,-0.039039,-0.017705,-0.002902,-0.014937,0.031546,-0.008590,-0.023857,-0.014054,-0.032486,-0.016127,-0.006829,0.010045,0.028259,0.022507,-0.009351,-0.032367,-0.010367,-0.022886,-0.008200,0.070938,0.022677,-0.004803,-0.024451,-0.023938,-0.018909,0.003604,-0.009259,0.074453,...,0.007414,0.007728,0.017345,0.005856,-0.052022,0.011893,0.016401,0.006308,0.000170,0.006218,0.006130,-0.011372,-0.013668,-0.001557,0.011272,-0.010685,0.013971,0.019989,-0.003609,-0.019990,-0.025714,-0.020191,-0.086680,-0.000248,0.001047,0.003517,-0.004713,-0.039130,-0.001972,0.004885,0.007225,0.006474,-0.100246,0.007575,-0.002297,0.022440,0.020384,0.076434,-0.026420,-0.048723
2,0,"[-0.021778336, -0.007887523, -0.012167458, 0.0...",-0.021778,-0.007888,-0.012167,0.026645,-0.047003,0.041111,0.032634,0.007721,-0.004673,0.017945,0.025572,0.020721,-0.029443,0.007282,-0.012125,-0.005743,-0.021941,-0.026845,-0.008875,0.024187,-0.035593,0.016917,0.015894,0.000627,-0.027061,-0.008227,-0.012238,0.004773,0.005052,0.086486,-0.012999,0.013584,0.025601,-0.023260,-0.010884,-0.021703,0.011610,0.058846,...,0.013974,-0.027056,-0.014892,0.007018,0.004775,0.003739,-0.004609,0.015453,0.005407,0.002417,0.010540,-0.009061,-0.004683,-0.022841,-0.009695,-0.015350,-0.000986,0.000599,-0.002325,0.004891,0.000679,-0.023098,-0.053412,0.000287,-0.038986,0.012374,-0.007347,0.018700,0.030905,0.007683,0.001130,0.012536,-0.078928,0.052752,-0.023307,-0.003699,0.008551,0.036900,0.011529,0.001560
3,0,"[0.021224087, 0.003118416, -0.035745192, 0.044...",0.021224,0.003118,-0.035745,0.044600,-0.007005,-0.017203,0.036646,0.000290,0.003102,0.027911,-0.044804,-0.008287,0.004489,-0.016696,0.003103,0.000320,-0.033806,-0.013790,-0.012040,-0.013854,0.008394,0.004658,0.036740,0.023992,0.018495,-0.019312,0.013242,0.006967,-0.012553,0.111057,-0.001287,0.014520,-0.001761,-0.015324,-0.004771,0.005732,0.008985,0.033945,...,-0.024249,-0.016567,0.023114,0.011763,-0.032777,-0.000594,0.011036,0.010664,-0.001417,-0.000932,-0.023270,0.015315,-0.021258,-0.056209,0.026135,0.011350,0.022573,-0.003763,-0.043058,-0.008284,0.011969,0.016680,-0.055033,-0.025068,-0.000159,0.027143,-0.007409,-0.054329,-0.008116,0.003405,-0.017899,-0.001394,-0.123789,0.027707,0.001508,-0.001378,0.013516,0.063901,-0.025079,-0.014923
4,0,"[-0.04263019, 0.028402044, -0.02406443, 0.0112...",-0.042630,0.028402,-0.024064,0.011293,-0.001849,-0.013165,-0.012895,-0.024676,-0.038230,0.021767,-0.044259,-0.010850,0.051784,0.020780,-0.019907,-0.025127,0.008325,-0.010708,-0.043764,-0.035868,-0.055030,0.012830,-0.006759,0.015732,-0.050231,-0.014943,0.002229,0.049738,-0.021219,0.078146,0.012240,0.015609,-0.011149,-0.043478,-0.064961,0.013925,0.008646,-0.008385,...,0.040676,-0.015430,-0.018999,-0.002594,-0.047164,0.051087,0.005464,-0.019100,0.007361,-0.

Will be attempting xgboost here.

In [ ]:
#use when RUNTIME DISCONNECTED AND ONLY NEEDED FOR XGBOOST IMPLEMENTATION
df3 = pd.read_pickle('/content/sample_data/ourpickle22.pkl')

In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# split data into X and y
X = df3.iloc[:, 2:]
Y = df3['labelML']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20,random_state=1,stratify=Y)

In [ ]:
X_train.shape

(17712, 300)

In [ ]:
X_test.shape

(4428, 300)

In [ ]:
y_train

8048     1
18797    1
10946    1
5222     0
14884    1
        ..
20212    1
13507    1
2992     0
18798    1
21719    1
Name: labelML, Length: 17712, dtype: int64

In [ ]:
from collections import Counter
Counter(y_test).values()

dict_values([3363, 1065])

In [ ]:
# Instantiate the XGBClassifier: xg_cl
xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10, seed=123)

# Fit the classifier to the training set
xg_cl.fit(X_train, y_train)

# Predict the labels of the test set: preds
preds = xg_cl.predict(X_test)

# Compute the accuracy: accuracy
accuracy = float(np.sum(preds == y_test)) / y_test.shape[0]
print("accuracy: %f" % (accuracy))

accuracy: 0.771906


In [ ]:
from sklearn.metrics import confusion_matrix
#confusion_matrix(y_true, y_pred)

tn, fp, fn, tp = confusion_matrix([0, 1, 0, 1], [1, 1, 1, 0]).ravel()
(tn, fp, fn, tp)

(0, 2, 1, 1)

Next step, keras NN model.